In [ ]:
# Ladataan tarvittavat kirjastot
import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import os 

In [ ]:
# Haetaan airbnb data ja tallennetaan se csv-tiedostoon
if os.path.exists('abnd_listings.csv'):
    df = pd.read_csv('abnd_listings.csv')
else:
    df = pd.read_csv('https://raw.githubusercontent.com/InfoTUNI/joda2022/master/koodiesimerkit/data.csv')
    df.to_csv('abnd_listings.csv')

In [ ]:
# Tarkastellaan dataa
df.info() # Tietojen tyypit 
pd.set_option('display.max_columns', None) # Muutetaan Panda:n asetusta, jotta kaikki sarakkeet näkyvät
df.head() # Näytetään viisi ensimmäistä riviä

In [ ]:
# Tarkastellaan aluksi tyhjien kenttien määriä
missing_values_count = df.isnull().sum()
missing_values_count = missing_values_count[missing_values_count > 0] # Näytetään vain ne sarakkeet, joissa on tyhjiä arvoja
print(f"Missing values count : \n{missing_values_count}") # Tulostetaan tyhjien kenttien määrät

In [ ]:
# Tarkastellaan tyhjiä arvoja sisältävien sarakkeiden suhteellista osuutta
missing_values_percent = (missing_values_count / (df.shape[0])) * 100
print(f"Missing values percent : \n{missing_values_percent}") # Tulostetaan tyhjien kenttien suhteelliset osuudet


In [ ]:
# Visualisoidaan tyhjien arvojen suhteelliset osuudet kasvavaan järjestykseen 

missing_values_percent = missing_values_percent.sort_values() # Järjestellään kasvavaan järjestykseen

# Visualisointi 
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 15))  # Säädä kokoa tarpeen mukaan
sns.barplot(x=missing_values_percent, y=missing_values_percent.index)
plt.title('Puuttuvien arvojen prosenttiosuus sarakkeittain')
plt.xlabel('Prosenttiosuus (%)')
plt.ylabel('Sarakkeet')
plt.show()

In [ ]:
# Poistetaan datasta ne sarakkeet joissa yli 50% tyhjiä arvoja 
index_null_over_50 = missing_values_percent[missing_values_percent > 50].index
df_cleaned = df.drop(index_null_over_50, axis=1) 

# Tarkastellaan uutta dataa
df_cleaned.info()   

In [ ]:
# Otetaan datasta osajoukko joka sisältää erillaisia datatyyppejä harjoitusta varten joissa tyhjiä arvoja

df_subset = df_cleaned[['host_response_time', 'host_response_rate', 'review_scores_rating']]
df_subset.info()

In [ ]:
# Käsitellään host_response_time saraketta

# Täytetään null arvot 'unknown' arvolla
df_subset.loc[:, 'host_response_time'] = df_subset['host_response_time'].fillna('unknown') 

# Visualisoidaan arvojen jakauma prosentuaalinen jakauma 
host_response_times_percent = (df_subset['host_response_time'].value_counts() / df_subset.shape[0]) * 100
host_response_times_percent = host_response_times_percent.sort_values() 

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 5))
sns.barplot(x=host_response_times_percent, y=host_response_times_percent.index)
plt.title('Asuntojen vastausajan jakauma')
plt.xlabel('Prosenttiosuus (%)')
plt.ylabel('Vastausaika')
plt.show()



In [ ]:
# Käsitellään host_response_rate saraketta

# Parsitaan '%' pois ja näiden rivien arvot numeeriseksi 
df_subset.loc[:, 'host_response_rate'] = df_subset['host_response_rate'].str.rstrip('%').astype('float') 

# Täytetään puuttuvat arvot mediaanilla 
mediaani =  df_subset['host_response_rate'].median()
# infer_objects(copy=False) tarvittiin koska fillna toiminnallisuus muuttuu tulevaisuudessa
df_subset['host_response_rate'] = df_subset['host_response_rate'].infer_objects(copy=False).fillna(mediaani) 

# # Visualisoidaan arvojen jakauma
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 5))
sns.histplot(df_subset['host_response_rate'], kde=True)
plt.title('Asuntojen vastausprosentin jakauma')
plt.xlabel('Vastausprosentti (%)')
plt.ylabel('Asuntojen määrä')
plt.show()


In [ ]:
# Käsitellään review_scores_rating saraketta

mediaani =  df_subset['review_scores_rating'].median()
df_subset.loc[:, 'review_scores_rating'] = df_subset['review_scores_rating'].fillna(mediaani)
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 5))
sns.histplot(df_subset['review_scores_rating'], kde=True)
plt.title('Asuntojen arvostelupisteiden jakauma')
plt.xlabel('Pisteet')
plt.ylabel('Asuntojen määrä')
plt.show()


